In [22]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [23]:
import re
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score

In [24]:
url = 'https://raw.githubusercontent.com/Shimix77/Diplomova_praca/main/lemma_sk.csv'
lemma_sk = pd.read_csv(url, header = 0, delimiter = ";") #skiprows=2
lemma_sk.drop_duplicates(subset ="Konkretny_tvar_slova", keep = "first", inplace = True)
lemma_sk.shape

(1176800, 3)

In [25]:
lemma_sk.head(6)

,Zakladny_tvar_slova,Konkretny_tvar_slova,Znacka
0,A,A,W
1,AB,AB,W
2,ABS,ABS,W
3,AC,AC,W
4,AD,AD,W
5,ADOS,ADOS,W


In [26]:
url = "https://raw.githubusercontent.com/Shimix77/Diplomova_praca/main/stopwords_sk.csv"
stopwords_sk = pd.read_csv(url, header = 0) #skiprows=2
stopwords_sk.shape

(418, 1)

In [27]:
stopwords_sk.head(6)

,stopwords_slovak
0,a
1,aby
2,aj
3,ak
4,aká
5,akáže


In [28]:
#train = pd.read_csv('datasource_ML.csv',delimiter=',')
train = pd.read_csv('Datasource_pre_ML - 50vs50.csv',delimiter=',')
train.shape

(1625, 7)

In [29]:
train.head(6)

,Politik,Strana,Relácia,Výrok,Rok,Mesiac,Hodnota
0,Eduard Heger,Oľano,Sobotné dialógy,"Rozbehli sme (na Ukrajine, pozn.) veľmi intenz...",2022,2,Neoveriteľné
1,Juraj Krúpa,Oľano,O 5 minút 12,My sme dali riadny priestor na medzirezortné p...,2022,2,Neoveriteľné
2,Andrej Danko,SNS,Na hrane,Ja som dostal 2× pozvánku do kongresu Spojenýc...,2022,3,Neoveriteľné
3,Jaroslav Naď,Oľano,Na hrane,"Už to nie je tak, ako za pána Danka, že dal na...",2022,3,Neoveriteľné
4,Jaroslav Naď,Oľano,Na hrane,Tí Rusi tam robia genocídu obyvateľov.,2022,3,Neoveriteľné
5,Róbert Kaliňák,SMER-SD,V politike,"Nielenže teraz urobili zmluvu o to, aby niekto...",2022,3,Neoveriteľné


In [30]:
#nastavenie datasetu na malé písmená
def lower_case(df):
    df['Výrok'] = df['Výrok'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    print(df['Výrok'].head())
    
lower_case(train)

0    rozbehli sme (na ukrajine, pozn.) veľmi intenz...
1    my sme dali riadny priestor na medzirezortné p...
2    ja som dostal 2× pozvánku do kongresu spojenýc...
3    už to nie je tak, ako za pána danka, že dal na...
4               tí rusi tam robia genocídu obyvateľov.
Name: Výrok, dtype: object


In [31]:
#očistenie textu
def cleaning_text(vyrok):
    vyroky = " ".join(filter(lambda x: x[0]!= '!' , vyrok.split()))
    #vyroky = re.sub(r'[^a-zA-Z]', ' ', vyroky)
    vyroky = re.sub(r'[^\w\s]', ' ', vyroky)
    vyroky = vyroky.split()
    vyroky = " ".join(vyroky)
    return vyroky

train['Výrok_opravený'] = train.Výrok.apply(cleaning_text)
train.head(6)

,Politik,Strana,Relácia,Výrok,Rok,Mesiac,Hodnota,Výrok_opravený
0,Eduard Heger,Oľano,Sobotné dialógy,"rozbehli sme (na ukrajine, pozn.) veľmi intenz...",2022,2,Neoveriteľné,rozbehli sme na ukrajine pozn veľmi intenzívne...
1,Juraj Krúpa,Oľano,O 5 minút 12,my sme dali riadny priestor na medzirezortné p...,2022,2,Neoveriteľné,my sme dali riadny priestor na medzirezortné p...
2,Andrej Danko,SNS,Na hrane,ja som dostal 2× pozvánku do kongresu spojenýc...,2022,3,Neoveriteľné,ja som dostal 2 pozvánku do kongresu spojených...
3,Jaroslav Naď,Oľano,Na hrane,"už to nie je tak, ako za pána danka, že dal na...",2022,3,Neoveriteľné,už to nie je tak ako za pána danka že dal na m...
4,Jaroslav Naď,Oľano,Na hrane,tí rusi tam robia genocídu obyvateľov.,2022,3,Neoveriteľné,tí rusi tam robia genocídu obyvateľov
5,Róbert Kaliňák,SMER-SD,V politike,"nielenže teraz urobili zmluvu o to, aby niekto...",2022,3,Neoveriteľné,nielenže teraz urobili zmluvu o to aby niekto ...


In [32]:
train_lemmatize_stopwords = train.Výrok_opravený.str.split()
list_of_konkretny_tvar = lemma_sk["Konkretny_tvar_slova"].to_list()
list_of_stopwords = stopwords_sk["stopwords_slovak"].to_list()
ListOfMyList = []

#def lemmatizing_stopwords_text():
a = 1177
for vyroky in train_lemmatize_stopwords:
    myList = []
    for word in vyroky:
        if word not in list_of_stopwords:
            if word in list_of_konkretny_tvar:
                myList.append(lemma_sk.loc[lemma_sk["Konkretny_tvar_slova"]==word,"Zakladny_tvar_slova"].iloc[0])
            else:
                myList.append(word)  
    myList = " ".join(myList)
    ListOfMyList.append(myList)

In [33]:
train['Vyrok_lematizovany_stopwords'] = pd.Series(ListOfMyList)
train = train.drop(['Výrok'], axis=1)
train = train.drop(['Výrok_opravený'], axis=1)
train.head(3)

,Politik,Strana,Relácia,Rok,Mesiac,Hodnota,Vyrok_lematizovany_stopwords
0,Eduard Heger,Oľano,Sobotné dialógy,2022,2,Neoveriteľné,rozbehnúť ukrajine pozn veľmi intenzívne progr...
1,Juraj Krúpa,Oľano,O 5 minút 12,2022,2,Neoveriteľné,dať riadny priestor medzirezortné pripomienkov...
2,Andrej Danko,SNS,Na hrane,2022,3,Neoveriteľné,dostať 2 pozvánka kongres spojený štát


In [34]:
train = train.rename({'Vyrok_lematizovany_stopwords': 'Výrok'}, axis=1)

In [35]:
#Odstránenie diakritiky

def cleaning_text(vyroky):
    vyroky = re.sub(r'[àáâãäå]', 'a', vyroky)
    vyroky = re.sub(r'[èéêë]', 'e', vyroky)
    vyroky = re.sub(r'[ìíîï]', 'i', vyroky)
    vyroky = re.sub(r'[òóôõö]', 'o', vyroky)
    vyroky = re.sub(r'[ùúûü]', 'u', vyroky)
    vyroky = re.sub(r'[ý]', 'y', vyroky)
    vyroky = re.sub(r'[ť]', 't', vyroky)
    vyroky = re.sub(r'[š]', 's', vyroky)
    vyroky = re.sub(r'[ď]', 'd', vyroky)
    vyroky = re.sub(r'[ž]', 'z', vyroky)
    vyroky = re.sub(r'[č]', 'c', vyroky)
    vyroky = re.sub(r'[ň]', 'n', vyroky)
    vyroky = re.sub(r'[ľ]', 'l', vyroky)
    vyroky = vyroky.split()
    vyroky = " ".join(vyroky)
    return vyroky 

train['Výrok'] = train.Výrok.apply(cleaning_text)

#Indexácia hodnôt
Hodnota_index={"Pravda": 1,"Zavádzajúce": 0,"Nepravda": 0,"Neoveriteľné": 0}
#Hodnota_index={"Pravda": 1,"Zavádzajúce": 2,"Nepravda": 3,"Neoveriteľné": 4}
data = [train]

for dataset in data:
    dataset['Hodnota'] = dataset['Hodnota'].map(Hodnota_index)

train.head(6)

,Politik,Strana,Relácia,Rok,Mesiac,Hodnota,Výrok
0,Eduard Heger,Oľano,Sobotné dialógy,2022,2,0,rozbehnut ukrajine pozn velmi intenzivne progr...
1,Juraj Krúpa,Oľano,O 5 minút 12,2022,2,0,dat riadny priestor medzirezortne pripomienkov...
2,Andrej Danko,SNS,Na hrane,2022,3,0,dostat 2 pozvanka kongres spojeny stat
3,Jaroslav Naď,Oľano,Na hrane,2022,3,0,pan danka dat spravodajsky sluzba sledovat odp...
4,Jaroslav Naď,Oľano,Na hrane,2022,3,0,rus robit genocida obyvatel
5,Róbert Kaliňák,SMER-SD,V politike,2022,3,0,nielenze teraz urobit zmluva niekto varit vych...


In [36]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x)
vyrok_vectors = vectorizer.fit_transform(train['Výrok'])
vyrok_vectors = vyrok_vectors.toarray()

y = train["Hodnota"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(vyrok_vectors, y, test_size = 0.20, random_state = 0)

In [38]:
#Naive Bayes
#classifier_np = GaussianNB()
classifier_np = MultinomialNB()
classifier_np.fit(X_train, y_train)
y_pred_np = classifier_np.predict(X_test)
np_score = accuracy_score(y_test, y_pred_np)
#Decision Tree
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)
y_pred_dt = classifier_dt.predict(X_test)
dt_score = accuracy_score(y_test, y_pred_dt)
#Logistic Regression
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)
y_pred_lr = classifier_lr.predict(X_test)
lr_score = accuracy_score(y_test, y_pred_lr)
#Random Forest
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)
y_pred_rf = classifier_rf.predict(X_test)
rf_score = accuracy_score(y_test, y_pred_rf)
#Supoort Vector Machine
classifier_svm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
classifier_svm.fit(X_train, y_train)
y_pred_svm = classifier_svm.predict(X_test)
svm_score = accuracy_score(y_test, y_pred_svm)


print('Naive Bayes Accuracy: ', str(np_score*100))
print('Decision Tree Accuracy: ', str(dt_score*100))
print('Logistic Regression Accuracy: ', str(lr_score*100))
print('Random Forest Accuracy: ', str(rf_score*100))
print("Support Vector Machine Accuracy: ",str(svm_score*100))

Naive Bayes Accuracy:  53.84615384615385
Decision Tree Accuracy:  81.84615384615384
Logistic Regression Accuracy:  54.46153846153846
Random Forest Accuracy:  75.38461538461539
Support Vector Machine Accuracy:  51.38461538461539
